# Prepare dataset of main process

In [1]:
path = '/Users/davide/Documents/universita/tesi/src/survival_process'
setwd(path)

require(data.table)
require(lubridate)

source('../preprocess/utils/minor.R')

Loading required package: data.table
Loading required package: lubridate

Attaching package: ‘lubridate’

The following objects are masked from ‘package:data.table’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year

The following object is masked from ‘package:base’:

    date



## Data loading

In [11]:
# Load data
load('../../data/dati_HF_sample.RData')
# Rename dataset
df = bla
rm(bla)

# load preprocessed dataset to select patients
load('../../data/preprocessed_data_ACE_2.RData')
sel_df = ACE_df
patients = unique(sel_df$id)

# load fpca scores
load('../../data/score_fpca_ACE.RData')

## Preprocessing

In [13]:
# rename features
df = rename_features(df)
# select rows relative to selected patients
df = df[id %in% patients]
# add time_event column
df[,'time_event' := data_studio_out - (data_rif_ev + make_difftime('days' = 366))]

In [14]:
# add age_in (before follow-up)
df = add_age_in(df,verbose = TRUE)
# add one year (time of follow-up):
df[,'age_in' := age_in+1]

[1] "Adding age_in column (age at t=0, i.e. at dismission from first hospitalization)"
  |======================================================================| 100%


In [16]:
# group by patient
new_df = df[,list(
    'sex' = first(sex),
    'age_in' = first(age_in),
    'time_event' = first(time_event),
    'status' = first(desc_studio_out)
  ),by=c("id")]

In [17]:
# add fpca scores
setorderv(new_df, c('id')) 
setorderv(score, c('id')) 

new_df[,'PC1' := score$PC1]
new_df[,'PC2' := score$PC2]

In [18]:
# change columns order
columns = c('id','sex','age_in','PC1','PC2','time_event','status')
new_df = new_df[,..columns]

In [19]:
# minors
new_df[,status:=ifelse(status== 'DECEDUTO',1,0)]
new_df[,time_event:=as.numeric(new_df$time_event)]

In [20]:
head(new_df)

id,sex,age_in,PC1,PC2,time_event,status
10000243,M,70,-40.851181,1.0158516,51,1
10000471,F,80,-41.405426,0.8888934,1058,1
10000717,M,73,-10.854551,-9.4426737,705,0
10003004,F,75,4.612457,-2.4369352,1658,1
10006065,M,69,61.216558,6.4277908,1950,0
10007000,F,84,-2.634210,3.4520296,238,1


In [21]:
save(new_df, file = '../../data/main_process_preprocessed_data_2.RData')
write.csv(new_df, file = '../../data/main_process_preprocessed_data_2.csv',row.names=FALSE)